In [ ]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import mysql.connector

## Taking Finalized Datasets and putting them into Pandas Dataframes

In [ ]:
weather = pd.read_csv('Berkeley_Weather_Final.csv')
co2 = pd.read_csv('owid-co2-data-final.csv')
crops = pd.read_csv('PSD_Grains_Final.csv')
coor = pd.read_csv('CountryCoord.csv')

## Sorting Dataframes by Country and Year

In [ ]:
weather_sorted = weather.sort_values(['Country', 'Year'])

In [ ]:
co2_sorted = co2.sort_values(['country', 'year'])

In [ ]:
crops_sorted = crops.sort_values(['Country_Name', 'Market_Year'])

In [ ]:
weather_sorted = weather_sorted.drop('Instances', axis = 1)

## Inserting Country Coordinates

In [ ]:
coor = coor.drop('Unnamed: 0', axis = 1)

In [ ]:
coordinates = coor.values.tolist()

In [ ]:
# Creating a connection to your MySQL Server 
# Put your MySQL Server/Workbench password
Coor_Info = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='your_password',
    database = 'global_country_info')

if (Coor_Info):
    print("Connection Successful")
else:
    print("Connection Failed")

Connection Successful


In [ ]:
# Create a cursor from the connection made
coor_cursor = Coor_Info.cursor() 
    
coor_statement = "INSERT INTO CountryLocation (Latitude, Longitude, Country) VALUES (%s, %s, %s)"

coor_cursor.executemany(coor_statement, coordinates)
  
Coor_Info.commit()
    
coor_cursor.execute("SELECT * FROM CountryLocation;")
coor_results_final = coor_cursor.fetchall() 

In [ ]:
Coor_DF = pd.DataFrame(coor_results_final, columns= ["country", "latitude", "longitude"]) 

In [ ]:
coor_cursor.close()
Coor_Info.close()

In [ ]:
Coor_DF

,country,latitude,longitude
0,Afghanistan,33.939110,67.709953
1,Albania,41.153332,20.168331
2,Algeria,28.033886,1.659626
3,Angola,-11.202692,17.873887
4,Argentina,-38.416097,-63.616672
...,...,...,...
131,Venezuela,6.423750,-66.589730
132,Vietnam,14.058324,108.277199
133,Yemen,15.552727,48.516388
134,Zambia,-13.133897,27.849332


## Join Temperature Data to CO2 data

In [ ]:
co2_sorted = co2_sorted.drop("Unnamed: 0", axis=1)

In [ ]:
weather_sorted.rename(columns = {'Country': 'country', 'Year': 'year'}, inplace = True)

In [ ]:
country_sorted_new = pd.merge(weather_sorted, co2_sorted, on = ['country', 'year'], how = 'left')

In [ ]:
country_sorted_new.isna().sum()

year                                       0
country                                    0
Yearly Average Temperature                 0
Yearly Average Temperature Uncertainty     0
iso_code                                  86
trade_co2                                 86
cement_co2                                86
coal_co2                                  86
flaring_co2                               86
gas_co2                                   86
oil_co2                                   86
other_industry_co2                        86
consumption_co2                           86
total_ghg                                 86
total_ghg_excluding_lucf                  86
methane                                   86
nitrous_oxide                             86
population                                86
gdp                                       86
primary_energy_consumption                86
dtype: int64

In [ ]:
country_sorted_new[country_sorted_new['country'] == 'Bhutan']= country_sorted_new[country_sorted_new['country'] == 'Bhutan'].fillna({'iso_code': 'BTN'}) 

In [ ]:
country_sorted_new[country_sorted_new['country'] == 'Oman'] = country_sorted_new[country_sorted_new['country'] == 'Oman'].fillna({'iso_code': 'OMN'}) 

In [ ]:
country_sorted_new[country_sorted_new['country'] == 'Botswana'] = country_sorted_new[country_sorted_new['country'] == 'Botswana'].fillna({'iso_code': 'BWA'}) 

In [ ]:
country_sorted_new[country_sorted_new['country'] == 'Namibia'] = country_sorted_new[country_sorted_new['country'] == 'Namibia'].fillna({'iso_code': 'NAM'}) 

In [ ]:
country_sorted_new[country_sorted_new['country'] == 'Lesotho'] = country_sorted_new[country_sorted_new['country'] == 'Lesotho'].fillna({'iso_code': 'LSO'}) 

## IMPUTATION OF DATA FOR COUNTRIES THAT DIDN'T START REPORTING IN 1960

In [ ]:
from fancyimpute import KNN

In [ ]:
numeric_cols = country_sorted_new.select_dtypes(include='number').columns.tolist()

In [ ]:
country_sorted_new[numeric_cols] = KNN(k=10).fit_transform(country_sorted_new[numeric_cols])

Imputing row 1/7208 with 0 missing, elapsed time: 4.878
Imputing row 101/7208 with 0 missing, elapsed time: 4.878
Imputing row 201/7208 with 0 missing, elapsed time: 4.878
Imputing row 301/7208 with 0 missing, elapsed time: 4.878
Imputing row 401/7208 with 0 missing, elapsed time: 4.878
Imputing row 501/7208 with 0 missing, elapsed time: 4.879
Imputing row 601/7208 with 0 missing, elapsed time: 4.879
Imputing row 701/7208 with 0 missing, elapsed time: 4.882
Imputing row 801/7208 with 15 missing, elapsed time: 4.884
Imputing row 901/7208 with 0 missing, elapsed time: 4.886
Imputing row 1001/7208 with 0 missing, elapsed time: 4.887
Imputing row 1101/7208 with 0 missing, elapsed time: 4.887
Imputing row 1201/7208 with 0 missing, elapsed time: 4.887
Imputing row 1301/7208 with 0 missing, elapsed time: 4.887
Imputing row 1401/7208 with 0 missing, elapsed time: 4.887
Imputing row 1501/7208 with 0 missing, elapsed time: 4.887
Imputing row 1601/7208 with 0 missing, elapsed time: 4.888
Imputing

In [ ]:
country_sorted_new.isna().sum()

year                                      0
country                                   0
Yearly Average Temperature                0
Yearly Average Temperature Uncertainty    0
iso_code                                  0
trade_co2                                 0
cement_co2                                0
coal_co2                                  0
flaring_co2                               0
gas_co2                                   0
oil_co2                                   0
other_industry_co2                        0
consumption_co2                           0
total_ghg                                 0
total_ghg_excluding_lucf                  0
methane                                   0
nitrous_oxide                             0
population                                0
gdp                                       0
primary_energy_consumption                0
dtype: int64

In [ ]:
country_sorted_new

,year,country,Yearly Average Temperature,Yearly Average Temperature Uncertainty,iso_code,trade_co2,cement_co2,coal_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,consumption_co2,total_ghg,total_ghg_excluding_lucf,methane,nitrous_oxide,population,gdp,primary_energy_consumption
0,1960.0,Afghanistan,13.985417,0.441833,AFG,4.825121,0.018,0.127,0.468075,6.331716,0.269,0.019286,52.630406,43.612099,26.255411,18.762165,6.944695,8996967.0,1.303325e+10,2.949846
1,1961.0,Afghanistan,14.064917,0.398083,AFG,2.799278,0.022,0.176,0.548903,7.603425,0.293,0.020133,33.225018,43.999025,26.648972,19.038970,7.049714,9169406.0,1.314629e+10,3.017879
2,1962.0,Afghanistan,13.768667,0.406167,AFG,5.166143,0.029,0.297,0.694566,9.074737,0.363,0.021839,54.138556,44.214932,26.863575,19.168252,7.099816,9351442.0,1.336763e+10,3.131269
3,1963.0,Afghanistan,15.033417,0.401250,AFG,5.396530,0.051,0.264,0.475554,10.127002,0.392,0.027291,59.306020,44.427530,27.077185,19.308069,7.149380,9543200.0,1.363030e+10,3.257176
4,1964.0,Afghanistan,13.084917,0.449417,AFG,2.222403,0.062,0.300,0.586721,18.013322,0.476,0.033849,21.192878,44.667891,27.308836,19.459534,7.211517,9744772.0,1.387050e+10,3.278518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203,2008.0,Zimbabwe,21.545583,0.433000,ZWE,1.575000,0.154,5.958,1.531992,4.544799,1.608,0.039210,9.295000,35.350000,24.150000,10.560000,5.970000,12379553.0,1.505022e+10,41.277000
7204,2009.0,Zimbabwe,21.377250,0.436417,ZWE,2.595000,0.270,3.559,1.526867,14.497649,1.648,0.036948,8.071000,36.270000,25.170000,10.920000,6.360000,12526964.0,1.627576e+10,41.112000
7205,2010.0,Zimbabwe,21.986250,0.409667,ZWE,1.497000,0.308,5.632,1.465790,12.835969,1.938,0.026987,9.375000,39.300000,28.080000,11.450000,6.770000,12697728.0,1.791838e+10,46.599000
7206,2011.0,Zimbabwe,21.602417,0.393417,ZWE,1.046000,0.382,6.101,1.452206,11.580721,3.261,0.041595,10.790000,42.040000,30.550000,11.900000,6.950000,12894323.0,1.955407e+10,51.260000


In [ ]:
country_data = country_sorted_new.values.tolist()

## Inserting Country Data into the Country Table

In [ ]:
# Creating a connection to your MySQL Server 
# Put your MySQL Server/Workbench password
Countries_Info = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='your_password',
    database = 'global_country_info')

if (Countries_Info):
    print("Connection Successful")
else:
    print("Connection Failed")

Connection Successful


In [ ]:
# Create a cursor from the connection made
country_cursor = Countries_Info.cursor() 

In [ ]:
country_statement = "INSERT INTO Countries (Year, Country, yearly_average_temperature, yearly_average_temperature_uncertainty, ISO_Code, Trade_co2, Cement_co2, Coal_co2, Flaring_co2, Gas_co2, Oil_co2, Other_industry_co2, Consumption_co2, Total_ghg, Total_ghg_excl_lucf, Methane, Nitrous_oxide, Population, GDP, Primary_Energy_Consumption) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

country_cursor.executemany(country_statement, country_data)
  
Countries_Info.commit()
    
country_cursor.execute("SELECT * FROM Countries;")
country_results_final = country_cursor.fetchall()

In [ ]:
# Using the countries results saved from the previous Query to insert
# Into a Pandas DataFrame to be used later
Country_DF = pd.DataFrame(country_results_final, columns= ["Record_ID", "Country", "Year", "ISO_Code", "Yearly_Average_Temperature", "Yearly_Average_Temperature_Uncertainty", "Methane", "Nitrous_oxide", "Total_ghg", "Total_ghg_excl_lucf", "Trade_co2", "Cement_co2", "Coal_co2", "Flaring_co2", "Gas_co2", "Oil_co2", "Other_industry_co2", "Consumption_co2", "Population", "GDP", "Primary Energy Consumption"])

In [ ]:
Country_DF

,Record_ID,Country,Year,ISO_Code,Yearly_Average_Temperature,Yearly_Average_Temperature_Uncertainty,Methane,Nitrous_oxide,Total_ghg,Total_ghg_excl_lucf,...,Cement_co2,Coal_co2,Flaring_co2,Gas_co2,Oil_co2,Other_industry_co2,Consumption_co2,Population,GDP,Primary Energy Consumption
0,1,Afghanistan,1960,AFG,13.98541667,0.441833333333333,18.76,6.94,43.61,26.26,...,0.018,0.127,0.468,6.332,0.269,0.019,52.630,8996967,13033254000,2.950
1,2,Afghanistan,1961,AFG,14.06491667,0.398083333333333,19.04,7.05,44.00,26.65,...,0.022,0.176,0.549,7.603,0.293,0.020,33.225,9169406,13146287000,3.018
2,3,Afghanistan,1962,AFG,13.76866667,0.406166666666667,19.17,7.10,44.21,26.86,...,0.029,0.297,0.695,9.075,0.363,0.022,54.139,9351442,13367634000,3.131
3,4,Afghanistan,1963,AFG,15.03341667,0.401250000000000,19.31,7.15,44.43,27.08,...,0.051,0.264,0.476,10.127,0.392,0.027,59.306,9543200,13630298000,3.257
4,5,Afghanistan,1964,AFG,13.08491667,0.449416666666667,19.46,7.21,44.67,27.31,...,0.062,0.300,0.587,18.013,0.476,0.034,21.193,9744772,13870504000,3.279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203,7204,Zimbabwe,2008,ZWE,21.54558333,0.433000000000000,10.56,5.97,35.35,24.15,...,0.154,5.958,1.532,4.545,1.608,0.039,9.295,12379553,15050220990,41.277
7204,7205,Zimbabwe,2009,ZWE,21.37725000,0.436416666666667,10.92,6.36,36.27,25.17,...,0.270,3.559,1.527,14.498,1.648,0.037,8.071,12526964,16275763662,41.112
7205,7206,Zimbabwe,2010,ZWE,21.98625000,0.409666666666667,11.45,6.77,39.30,28.08,...,0.308,5.632,1.466,12.836,1.938,0.027,9.375,12697728,17918380597,46.599
7206,7207,Zimbabwe,2011,ZWE,21.60241667,0.393416666666667,11.90,6.95,42.04,30.55,...,0.382,6.101,1.452,11.581,3.261,0.042,10.790,12894323,19554073776,51.260


In [ ]:
country_cursor.close()
Countries_Info.close()

## Preparing columns to integrate the Record_ID column into the PSD dataframe 

In [ ]:
PrimaryAutoGrains = Country_DF[['Record_ID', 'Country', 'Year']]

In [ ]:
PrimaryAutoGrains.rename(columns = {'Country':'Country_Name', 'Year': 'Market_Year'}, inplace = True)

C:\Users\rho25\AppData\Local\Temp\ipykernel_17360\2006266999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PrimaryAutoGrains.rename(columns = {'Country':'Country_Name', 'Year': 'Market_Year'}, inplace = True)


In [ ]:
PrimaryAutoGrains

,Record_ID,Country_Name,Market_Year
0,1,Afghanistan,1960
1,2,Afghanistan,1961
2,3,Afghanistan,1962
3,4,Afghanistan,1963
4,5,Afghanistan,1964
...,...,...,...
7203,7204,Zimbabwe,2008
7204,7205,Zimbabwe,2009
7205,7206,Zimbabwe,2010
7206,7207,Zimbabwe,2011


In [ ]:
crops_sorted_new = pd.merge(PrimaryAutoGrains, crops_sorted , on = ['Country_Name', 'Market_Year'], how = 'left')

In [ ]:
numeric_columns = crops_sorted_new.select_dtypes(include=['number']).columns

In [ ]:
numeric_columns

Index(['Record_ID', 'Market_Year', 'Unnamed: 0', 'Commodity_Code',
       'Area Harvested', 'Beginning Stocks', 'Ending Stocks', 'Exports',
       'FSI Consumption', 'Feed Dom. Consumption', 'Imports',
       'Milling Rate (.9999)', 'Production', 'Rough Production', 'TY Exports',
       'TY Imp. from U.S.', 'TY Imports'],
      dtype='object')

In [ ]:
crops_sorted_new[numeric_columns] = crops_sorted_new[numeric_columns].fillna(0)
crops_sorted_new['Commodity_Description'] = crops_sorted_new["Commodity_Description"].fillna("Empty")

In [ ]:
crops_sorted_new.isna().sum()

Record_ID                0
Country_Name             0
Market_Year              0
Unnamed: 0               0
Commodity_Code           0
Commodity_Description    0
Area Harvested           0
Beginning Stocks         0
Ending Stocks            0
Exports                  0
FSI Consumption          0
Feed Dom. Consumption    0
Imports                  0
Milling Rate (.9999)     0
Production               0
Rough Production         0
TY Exports               0
TY Imp. from U.S.        0
TY Imports               0
dtype: int64

In [ ]:
crops_sorted_new.drop(['Country_Name', 'Market_Year', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
crops_data = crops_sorted_new.values.tolist()

## Inserting PSD Data into the Grains Table

In [ ]:
# Creating a connection to your MySQL Server 
# Put your MySQL Server/Workbench password
Grains_Info = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='your_password',
    database = 'global_country_info')

if (Grains_Info):
    print("Connection Successful")
else:
    print("Connection Failed")

Connection Successful


In [ ]:
# Create a cursor from the connection made
grains_cursor = Grains_Info.cursor() 
    
grains_statement = "INSERT INTO Grains (Record_ID, Commodity_ID, Commodity_Description, Area_Harvested_HA, Beginning_Stocks_MT, Ending_Stocks_MT, Exports_MT, FSI_Consumption_MT, Feed_Dom_Consumption_MT, Imports_MT, Milling_Rate_MT, Production_MT ,Rough_Production_MT, TY_Exports_MT, TY_Imports_from_US_MT, TY_Imports_MT) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                  
grains_cursor.executemany(grains_statement, crops_data)
  
Grains_Info.commit()
    
grains_cursor.execute("SELECT * FROM Grains;")
grains_results_final = grains_cursor.fetchall() 
    

In [ ]:
# Using the grain_results saved from the previous Query to insert
# Into a Pandas DataFrame to be used later
Grain_DF = pd.DataFrame(grains_results_final, columns= ["Grains_Record_ID", "Record_ID","Commodity_ID", "Commodity_Description", "Area_Harvested_HA", "Beginning_Stocks_MT", "Ending_Stocks_MT", "Imports_MT", "Exports_MT", "Feed_Dom_Consumption_MT", "FSI_Consumption_MT", "Production_MT", "TY_Imports_MT", "TY_Imports_from_US_MT", "TY_Exports_MT", "Milling_Rate_MT", "Rough_Production_MT"])

In [ ]:
grains_cursor.close()
Grains_Info.close()

In [ ]:
Grain_DF.set_index("Grains_Record_ID", inplace=True)

In [ ]:
Grain_DF

,Record_ID,Commodity_ID,Commodity_Description,Area_Harvested_HA,Beginning_Stocks_MT,Ending_Stocks_MT,Imports_MT,Exports_MT,Feed_Dom_Consumption_MT,FSI_Consumption_MT,Production_MT,TY_Imports_MT,TY_Imports_from_US_MT,TY_Exports_MT,Milling_Rate_MT,Rough_Production_MT
Grains_Record_ID,,,,,,,,,,,,,,,,
1,1,410000,Wheat,2230000,0,0,51000,0,0,2330000,2279000,51000,46000,0,0,0
2,1,422110,"Rice, Milled",207000,0,0,0,0,0,0,203000,0,0,0,6500000,313000
3,1,430000,Barley,350000,0,0,0,0,0,378000,378000,0,0,0,0,0
4,1,440000,Corn,500000,0,0,0,0,0,700000,700000,0,0,0,0,0
5,2,410000,Wheat,2230000,0,0,15000,0,0,2294000,2279000,15000,15000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27489,7208,410000,Wheat,10000,52000,36000,250000,0,0,300000,34000,250000,0,0,0,0
27490,7208,430000,Barley,9000,4000,4000,15000,0,5000,61000,51000,15000,0,0,0,0
27491,7208,440000,Corn,1281000,25000,15000,600000,0,80000,1529000,999000,700000,0,0,0,0


## Checking for consistencies between the original datasets and finalized integrated database

In [ ]:
finalized = pd.read_csv('IntegratedData.csv')

In [ ]:
testIntegrate = (finalized.loc[(finalized['W_Country'] == 'Brazil') & (finalized['W_Year'] == 2010)])

In [ ]:
testco2 = (co2_sorted.loc[(co2_sorted['country'] == 'Brazil') & (co2_sorted['year'] == 2010)])

In [ ]:
testCrops= (crops_sorted.loc[(crops_sorted['Country_Name'] == 'Brazil') & (crops_sorted['Market_Year'] == 2010)])

## Comparing Commodities data

In [ ]:
testCrops[['Commodity_Description', 'Area Harvested', 'Imports']]

,Commodity_Description,Area Harvested,Imports
652,Wheat,2150000.0,6693000.0
5923,"Rice, Milled",2833000.0,632000.0
11038,Barley,88000.0,324000.0
14109,Corn,13800000.0,791000.0
18850,Rye,2000.0,0.0
19932,Oats,154000.0,0.0
23534,Sorghum,817000.0,1000.0


In [ ]:
testIntegrate[['Commodity_Description', 'Area_Harvested_HA', 'Imports_MT']]

,Commodity_Description,Area_Harvested_HA,Imports_MT
3379,Wheat,2150000,6693000
3380,"Rice, Milled",2833000,632000
3381,Barley,88000,324000
3382,Corn,13800000,791000
3383,Rye,2000,0
3384,Oats,154000,0
3385,Sorghum,817000,1000


## Comparing CO2 Data

In [ ]:
testco2[['population', 'gdp', 'methane']]

,population,gdp,methane
876,195713637.0,2.816706e+12,421.5


In [ ]:
testIntegrate[['Population', 'GDP', 'Methane']]

,Population,GDP,Methane
3379,195713637,2816706146073,421.5
3380,195713637,2816706146073,421.5
3381,195713637,2816706146073,421.5
3382,195713637,2816706146073,421.5
3383,195713637,2816706146073,421.5
3384,195713637,2816706146073,421.5
3385,195713637,2816706146073,421.5
